In [1]:
import tkinter as tk  # Import tkinter for the main interface
from tkinter import messagebox  # Import messagebox for error messages
import cv2  # OpenCV for video capture and face recognition
import pickle  # For loading face encodings
import csv  # For logging recognized faces
import time  # For managing time-related functionalities
import datetime  # For handling timestamps
import numpy as np  # For handling arrays and face distances
import face_recognition  # For face recognition functionalities


In [2]:

class FaceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition Attendance System")
        self.root.geometry("800x600")

        # Frame for information display (Status and last recognized person)
        self.info_frame = tk.Frame(root)
        self.info_frame.pack(pady=10)

        # Status label
        self.status_label = tk.Label(self.info_frame, text="Status: Not Running", font=("Helvetica", 12))
        self.status_label.pack(pady=10)

        # Last recognized name label
        self.name_label = tk.Label(self.info_frame, text="Last Recognized: None", font=("Helvetica", 12))
        self.name_label.pack(pady=10)

        # Frame for buttons
        self.frame_buttons = tk.Frame(root)
        self.frame_buttons.pack(pady=10)

        # Start button
        self.start_button = tk.Button(self.frame_buttons, text="Start", command=self.start_recognition, bg="#4CAF50", fg="white", width=15)
        self.start_button.grid(row=0, column=0, padx=10)

        # Stop button
        self.stop_button = tk.Button(self.frame_buttons, text="Stop", command=self.stop_recognition, bg="#f44336", fg="white", width=15, state=tk.DISABLED)
        self.stop_button.grid(row=0, column=1, padx=10)

        # Load face encodings and IDs
        with open("EncodeFile.p", 'rb') as file:
            encodeListKnownWithIds = pickle.load(file)
        self.encodeListKnown, self.studentIds = encodeListKnownWithIds[0], encodeListKnownWithIds[1]

        # Dictionary to track last recognized time for each person
        self.last_recognition_time = {name: 0 for name in self.studentIds}
        self.recognition_cooldown = 5 * 60  # 5 minutes in seconds
        self.output_file_path = 'C:/Users/shaba/Documents/recognized_faces.csv'  # Update this path

        # Flag for controlling webcam feed
        self.stop_flag = False
        self.cap = None  # Initialize webcam capture

        # Ensure webcam releases on close
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_recognition(self):
        self.stop_flag = False
        self.stop_button.config(state=tk.NORMAL)
        self.start_button.config(state=tk.DISABLED)
        self.status_label.config(text="Status: Running")
        self.cap = cv2.VideoCapture(0)  # Start webcam feed
        self.update_video_feed()  # Start video feed on the main thread

    def stop_recognition(self):
        self.stop_flag = True
        self.stop_button.config(state=tk.DISABLED)
        self.start_button.config(state=tk.NORMAL)
        self.status_label.config(text="Status: Stopped")
        if self.cap:
            self.cap.release()  # Release webcam when stopped
            self.cap = None
        cv2.destroyAllWindows()  # Close any OpenCV windows

    def update_video_feed(self):
        if self.stop_flag:
            return

        success, img = self.cap.read()  # Read frame from webcam
        if not success:
            print("Error: Could not read frame.")
            return

        imgRgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert frame to RGB for face recognition
        faceLocations = face_recognition.face_locations(imgRgb)  # Detect faces
        encodingsCurrentFrame = face_recognition.face_encodings(imgRgb, faceLocations)  # Get face encodings

        for encodeFace, faceLoc in zip(encodingsCurrentFrame, faceLocations):
            matches = face_recognition.compare_faces(self.encodeListKnown, encodeFace)  # Compare faces
            faceDist = face_recognition.face_distance(self.encodeListKnown, encodeFace)  # Get face distances
            matchIndex = np.argmin(faceDist)  # Get best match

            if matches[matchIndex]:  # If match found
                name = self.studentIds[matchIndex]  # Get person's name
                current_time = time.time()

                # If recognized after cooldown period, log it
                if current_time - self.last_recognition_time[name] >= self.recognition_cooldown:
                    self.last_recognition_time[name] = current_time
                    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')  # Timestamp format

                    # Write recognized name and timestamp to CSV
                    try:
                        with open(self.output_file_path, mode='a', newline='') as csv_file:
                            fieldnames = ['Name', 'Timestamp']
                            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                            writer.writerow({'Name': name, 'Timestamp': timestamp})
                            print(f"Data logged: {name} at {timestamp}")  # Debug message
                    except Exception as e:
                        print(f"Error writing to CSV: {e}")

                    # Update current recognized name and UI label for last recognized person
                    self.name_label.config(text=f"Last Recognized: {name}")

                # Draw a rectangle around the face
                y1, x2, y2, x1 = faceLoc  # Get coordinates of the face
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw rectangle around face

                # Draw name near the recognized face
                cv2.putText(img, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)  # Display name

        # Display the image using OpenCV window
        cv2.imshow("Video Feed", img)  # Show the webcam feed in a window

        # Update the feed every 10 ms
        self.root.after(10, self.update_video_feed)

    def on_closing(self):
        self.stop_recognition()  # Ensure webcam is released
        self.root.destroy()  # Close the Tkinter window

# Run the Tkinter application
root = tk.Tk()
app = FaceRecognitionApp(root)
root.mainloop()


Data logged: shabarri at 2024-10-10 10:14
Data logged: suriya at 2024-10-10 10:14
Data logged: vijay at 2024-10-10 10:14
